© Alexander Jüstel, Fraunhofer IEG, Institution for Energy Infrastructures and Geothermal Systems, RWTH Aachen University,
GNU Lesser General Public License v3.0


This notebook was created by Alexander Jüstel and is part of the DGE Rollout Heat Demand Mapping project carried out by students of RWTH Aachen University within the framework of their master thesis. 

# Statistical Evaluation of the residential Heat demand data

This notebook illustrates the statistical evaluation on a national scale for the residential heat demand using `rasterstats` and `pandas`.


# Calculating zonal statistics for residential heat demand

The zonal statistics will be calculated using a shape file containing one polygon for each country and the raster containing the residential heat demand.

# Importing Libraries

In [1]:
# Importing Libraries
from rasterstats import zonal_stats
import pandas as pd
import geopandas as gpd

In [2]:
# Calculating zonal statistics
stats_res_heat = zonal_stats("../../../data/heat_demand_mapping/shape/Interreg_NWE_Countries_3034.shp", 
                             "../../../data/heat_demand_mapping/tif_results/NWE_HD_res.tif",
                             stats="count min mean max median sum std",
                             geojson_out=True)


## Storing the properties in a list 

All Properties will be stored in a list. The name of the countries will be stored as well to identify the data set later on. The values for Scotland will be set to zero as no residential heat demand was reported for this country. 

In [3]:
# Setting all values for Scotland to 0
stats_res_heat[4]['properties']['min'] = 0
stats_res_heat[4]['properties']['max'] = 0
stats_res_heat[4]['properties']['mean'] = 0
stats_res_heat[4]['properties']['count'] = 0
stats_res_heat[4]['properties']['sum'] = 0
stats_res_heat[4]['properties']['std'] = 0
stats_res_heat[4]['properties']['median'] = 0
stats_res_heat[4]['properties']

OrderedDict([('NAME_LATN', 'Scotland'),
             ('NUTS_NAME', 'Scotland'),
             ('area2', 74237),
             ('min', 0),
             ('max', 0),
             ('mean', 0),
             ('count', 0),
             ('sum', 0),
             ('std', 0),
             ('median', 0)])

In [4]:
# Creating list of properties dicts
properties_res_heat = [stats_res_heat[i]['properties'] for i in range(len(stats_res_heat))]
properties_res_heat[:2]

[OrderedDict([('NAME_LATN', 'France'),
              ('NUTS_NAME', 'France'),
              ('area2', 260768),
              ('min', 5.461e-12),
              ('max', 30381.642517981003),
              ('mean', 54.015635044222925),
              ('count', 4937110),
              ('sum', 266681131.93318346),
              ('std', 148.98699384143208),
              ('median', 13.026429339869845)]),
 OrderedDict([('NAME_LATN', 'Great Britain'),
              ('NUTS_NAME', 'Great Britain'),
              ('area2', 154556),
              ('min', 1.148e-12),
              ('max', 4204.1344217450105),
              ('mean', 120.83539584932839),
              ('count', 2652945),
              ('sum', 320569659.2414965),
              ('std', 168.33352559453797),
              ('median', 30.452353596783063)])]

## Creating a Dataframe from dicts

A dataframe containing all statistical values is created. 

In [5]:
# Creating DataFrame from list of dicts
df_res_heat = pd.DataFrame(properties_res_heat, index=range(9))

# Creating a column called 'Region'
df_res_heat['Region'] = df_res_heat['NAME_LATN']



## Cumulated Heat demand of the entire Interreg NWE region

The cumulated heat demand is equal to the `sum` of all country values in `MWh`converted to `TWh`. In addition, the column `sum` is equal to the `Combined/Total HD [TWh]`. The `Share of Total HD [%]` is the `Combined/Total HD [TWh]` divided by the cumulated heat demand and converted to `TWh`.

In [6]:
# Creating a new column called 'Residential HD [TWh]'
df_res_heat['Residential HD [TWh]'] = df_res_heat['sum']/1e6

# Calculating the residential cumulated heat demand of the Interreg NWE Region
df_res_heat_cumulated_hd = round(sum(df_res_heat['Residential HD [TWh]']),1)

# Calculating the share of national heat demand to the residential heat demand of the NWE Region
df_res_heat['Share of Residential HD [%]'] = df_res_heat['Residential HD [TWh]']/df_res_heat_cumulated_hd*100

print(str(df_res_heat_cumulated_hd) + ' TWh')

1129.3 TWh


## Calculating the area that is covered by heat demand squares

The are that is covered by heat is equal to the `count` of pixels of the raster times the resolution (100 m * 100 m) times the conversion factor from m2 to km2. In addition, the `Average of heated area total [MWh/ha]` is equal to `mean`. 

In [7]:
# Calculating the area that is covered by heat demand squares
df_res_heat['Area of heat demand [km2]'] = df_res_heat['count']*100*100*1e-6

# Calculating the total Interreg NWE region where a heat demand was calculated
total_area_of_heat_demand = round(sum(df_res_heat['Area of heat demand [km2]']))

# Creating a new column called 'Average of heated area residential [MWh/ha]'
df_res_heat['Average of heated area of Residential HD [MWh/ha]'] = df_res_heat['mean']


print(str(total_area_of_heat_demand) + ' km2')
df_res_heat

124161 km2


,NAME_LATN,NUTS_NAME,area2,min,max,mean,count,sum,std,median,Region,Residential HD [TWh],Share of Residential HD [%],Area of heat demand [km2],Average of heated area of Residential HD [MWh/ha]
0,France,France,260768,5.461000e-12,30381.642518,54.015635,4937110,2.666811e+08,148.986994,13.026429,France,266.681132,23.614729,49371.10,54.015635
1,Great Britain,Great Britain,154556,1.148000e-12,4204.134422,120.835396,2652945,3.205697e+08,168.333526,30.452354,Great Britain,320.569659,28.386581,26529.45,120.835396
2,Ireland,Ireland,65150,1.257709e-08,1544.301183,48.324403,456911,2.207995e+07,100.918000,7.736619,Ireland,22.079951,1.955189,4569.11,48.324403
3,Netherlands,Netherlands,26826,2.054444e-09,1827.180414,122.848967,425033,5.221486e+07,144.075270,58.634607,Netherlands,52.214865,4.623649,4250.33,122.848967
4,Scotland,Scotland,74237,0.000000e+00,0.000000,0.000000,0,0.000000e+00,0.000000,0.000000,Scotland,0.000000,0.000000,0.00,0.000000
5,Schweiz/Suisse/Svizzera,Schweiz/Suisse/Svizzera,38663,8.236287e-08,2825.353053,130.081813,363676,4.730763e+07,174.085592,65.788687,Schweiz/Suisse/Svizzera,47.307633,4.189111,3636.76,130.081813
6,Luxembourg,Luxembourg,2421,1.847969e-06,3428.203874,97.522674,49022,4.780757e+06,250.827789,3.987962,Luxembourg,4.780757,0.423338,490.22,97.522674
7,Germany,Germany,136800,1.108942e-09,6696.231562,122.989015,2767313,3.403491e+08,178.203988,43.175231,Germany,340.349101,30.138059,27673.13,122.989015
8,Belgium,Belgium,28595,2.934128e-09,11169.310656,98.556712,764091,7.530630e+07,181.653919,36.281220,Belgium,75.306297,6.668405,7640.91,98.556712


In [8]:
# Calculating the average heat demand of the heated area 
average_of_heated_area_total = sum(df_res_heat['mean'])/9

print(average_of_heated_area_total)

88.35273492058047


In [9]:
# Loading the Interreg NWE Region as Shape File
gdf_interreg_nwe = gpd.read_file("../../../data/heat_demand_mapping/shape/Interreg_NWE_Countries_3034.shp")

# Adding the area im m2 to the gdf
gdf_interreg_nwe['area'] = gdf_interreg_nwe.area

# Converting the area in m2 to km2
gdf_interreg_nwe['Area (planimetric) [km2]'] = round(gdf_interreg_nwe['area']*1e-6)

# Changing the data type of the column to int
gdf_interreg_nwe['Area (planimetric) [km2]'] = gdf_interreg_nwe['Area (planimetric) [km2]'].astype(int)

# Calculating the total planimetric area of the Interreg NWE Region
gdf_interreg_nwe_total_area_planimetric = sum(gdf_interreg_nwe['Area (planimetric) [km2]'])

# Adding the ellipsoidal areas manually to the DataFrame, values were calculated using QGIS
gdf_interreg_nwe['Area (ellipsoidal) [km2]']= [279030,
                                               165486,
                                               69738,
                                               28755,
                                               78684,
                                               41263,
                                               2596,
                                               146607,  
                                               30665]  

# Calculating the total ellipsoidal area of the Interreg NWE Region
gdf_interreg_nwe_total_area_ellipsoid = sum(gdf_interreg_nwe['Area (ellipsoidal) [km2]'])

# Calculating the share of each country to the total ellipsoidal Interreg NWE Region
gdf_interreg_nwe['Share of Total Area [%]'] = gdf_interreg_nwe['Area (ellipsoidal) [km2]']/gdf_interreg_nwe_total_area_ellipsoid*100

print(str(gdf_interreg_nwe_total_area_planimetric) + ' km2')
print(str(gdf_interreg_nwe_total_area_ellipsoid) + ' km2')
gdf_interreg_nwe

788016 km2
842824 km2


,NAME_LATN,NUTS_NAME,area2,geometry,area,Area (planimetric) [km2],Area (ellipsoidal) [km2],Share of Total Area [%]
0,France,France,260768,"MULTIPOLYGON (((2930641.617 2531547.039, 29306...",2.607676e+11,260768,279030,33.106556
1,Great Britain,Great Britain,154556,"MULTIPOLYGON (((2874711.281 2699278.506, 28747...",1.545561e+11,154556,165486,19.634704
2,Ireland,Ireland,65150,"MULTIPOLYGON (((2651794.967 2994393.242, 26518...",6.514953e+10,65150,69738,8.274325
3,Netherlands,Netherlands,26826,"MULTIPOLYGON (((3667957.194 2938854.305, 36680...",2.682582e+10,26826,28755,3.411744
4,Scotland,Scotland,74237,"MULTIPOLYGON (((3075395.957 3245560.731, 30753...",7.423735e+10,74237,78684,9.335757
5,Schweiz/Suisse/Svizzera,Schweiz/Suisse/Svizzera,38663,"POLYGON ((3963266.327 2322163.044, 3963280.662...",3.866341e+10,38663,41263,4.895803
6,Luxembourg,Luxembourg,2421,"POLYGON ((3733429.004 2606067.737, 3733284.924...",2.421410e+09,2421,2596,0.308012
7,Germany,Germany,136800,"MULTIPOLYGON (((3954764.925 2330292.165, 39546...",1.368000e+11,136800,146607,17.394735
8,Belgium,Belgium,28595,"MULTIPOLYGON (((3496569.868 2727532.821, 34976...",2.859470e+10,28595,30665,3.638363


## Calculating Average total heat demand

In [10]:
# Calculating the average residential heat demand including not heated areas
average_total_heat_demand = df_res_heat_cumulated_hd/gdf_interreg_nwe_total_area_ellipsoid * 1e4

print(str(average_total_heat_demand)+' MWh/ha')

13.399001452260494 MWh/ha


## Merge DataFrames

The two DataFrames are now being merged. In addition, a row for the values of the entire Interreg NWE Region is added.

In [11]:
# Merging DataFrames
gdf_res_heat = pd.merge(df_res_heat, gdf_interreg_nwe, 
                        on=['NUTS_NAME', 'area2'], 
                        how='left')

# Calculating the average HD demand per hectare including areas that are not heated 
gdf_res_heat['Average of Residential HD [MWh/ha]'] = gdf_res_heat['Residential HD [TWh]']*1e6/gdf_res_heat['Area (ellipsoidal) [km2]']/100

# Calculating the share of heated heat demand area to the total area
gdf_res_heat['Share of heated heat demand area [%]'] = gdf_res_heat['Area of heat demand [km2]']/gdf_res_heat['Area (ellipsoidal) [km2]']*100

# Creating a dict summarizing the the calculated values for the entire Interreg NWE Region
total_interreg = {'Region': 'Interreg NWE', 
                  'Area (planimetric) [km2]': gdf_interreg_nwe_total_area_planimetric,
                  'Area (ellipsoidal) [km2]': gdf_interreg_nwe_total_area_ellipsoid,
                  'Share of Total Area [%]': 100, 
                  'Residential HD [TWh]': df_res_heat_cumulated_hd,
                  'Share of Residential HD [%]': 100, 
                  'Average of Residential HD [MWh/ha]': average_total_heat_demand,
                  'Average of heated area of Residential HD [MWh/ha]': average_of_heated_area_total, 
                  'Area of heat demand [km2]': total_area_of_heat_demand,
                  'Share of heated heat demand area [%]': total_area_of_heat_demand/gdf_interreg_nwe_total_area_ellipsoid*100}

# Appending the row to the DataFrame
gdf_res_heat = gdf_res_heat.append(total_interreg,ignore_index=True)

# Selecting columns
gdf_res_heat = gdf_res_heat[['Region', 'Area (planimetric) [km2]', 
                             'Area (ellipsoidal) [km2]', 
                             'Share of Total Area [%]', 
                             'Residential HD [TWh]', 
                             'Share of Residential HD [%]', 
                             'Average of Residential HD [MWh/ha]', 
                             'Average of heated area of Residential HD [MWh/ha]', 
                             'Area of heat demand [km2]',
                             'Share of heated heat demand area [%]']].sort_values(by='Residential HD [TWh]', ascending=False).reset_index().drop('index', axis=1)
gdf_res_heat.round(decimals=1)#.style.applymap(lambda x: "background-color: red" if x>0 else "background-color: green")

,Region,Area (planimetric) [km2],Area (ellipsoidal) [km2],Share of Total Area [%],Residential HD [TWh],Share of Residential HD [%],Average of Residential HD [MWh/ha],Average of heated area of Residential HD [MWh/ha],Area of heat demand [km2],Share of heated heat demand area [%]
0,Interreg NWE,788016,842824,100.0,1129.3,100.0,13.4,88.4,124161.0,14.7
1,Germany,136800,146607,17.4,340.3,30.1,23.2,123.0,27673.1,18.9
2,Great Britain,154556,165486,19.6,320.6,28.4,19.4,120.8,26529.4,16.0
3,France,260768,279030,33.1,266.7,23.6,9.6,54.0,49371.1,17.7
4,Belgium,28595,30665,3.6,75.3,6.7,24.6,98.6,7640.9,24.9
5,Netherlands,26826,28755,3.4,52.2,4.6,18.2,122.8,4250.3,14.8
6,Schweiz/Suisse/Svizzera,38663,41263,4.9,47.3,4.2,11.5,130.1,3636.8,8.8
7,Ireland,65150,69738,8.3,22.1,2.0,3.2,48.3,4569.1,6.6
8,Luxembourg,2421,2596,0.3,4.8,0.4,18.4,97.5,490.2,18.9
9,Scotland,74237,78684,9.3,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
gdf_res_heat.round(decimals=1).to_csv('Statistics_Residential_Heat_Demand_NUTS0.csv', index=False)